# How Prevalent is Transitivity-Failure in Bayesian Confirmation?

This notebook corresponds to the following paper:

https://www.journals.uchicago.edu/doi/abs/10.1086/731830?journalCode=bjps

## Importing Packages and Setup

In [4]:
import numpy as np
import pandas as pd
import polars as pl
from scipy import stats
from itertools import chain, combinations
from IPython.display import Math, display
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning) 

ModuleNotFoundError: No module named 'numpy'

## Defining Functions for Powerset and Probabilities

In [50]:
# Powerset function (without the empty set)
def powerset(s):
    return chain.from_iterable(combinations(s, r) for r in range(1,len(s)+1))

# Function that calculates all marginal and joint probabilities for a set of probability distributions, given as matrix
def probabilities(matrix):

    # Size of matrix, number of random variables
    length, width = matrix.shape
    num_variables = np.log2(width).astype(int)
    
    # Create matrix with all Boolean combinations
    binmat = np.array([np.fromstring(np.binary_repr(i,num_variables), dtype=np.uint8)==49 for i in range(0,width)])

    # Pick all probabilities from matrix
    result = [pd.Series(np.sum(np.all(binmat[:,i],axis=1)*matrix,axis = 1), name = f"P{i}") for i in powerset(range(num_variables))]

    return pl.DataFrame(result)


## Simulating Probability Functions

In [111]:
# Number of variables and distributions
number_variables = 3
number_distributions = 1000000

# Sampling from Dirichlet distribution, calculating relevant probabilities
matrix = pl.DataFrame(np.random.dirichlet(np.ones(2**number_variables), number_distributions))
df = probabilities(matrix)

## Results for Bayesian Confirmation

In [112]:
### Some useful variables

# Defining antecedent for transitivity
antecedent = (df["P(0, 1)"] > df["P(0,)"]*df["P(1,)"]) & (df["P(1, 2)"] > df["P(1,)"]*df["P(2,)"])
ante_mean = antecedent.mean()

# Defining antecedent for cumulative transitivity
antecedent_cum = (df["P(0, 1)"] > df["P(0,)"]*df["P(1,)"]) & (df["P(0, 1, 2)"] > df["P(0, 1)"]*df["P(2,)"]) 
antecedent_cum_mean = antecedent_cum.mean()

# Defining antecedent for rational monotonicity
antecedent_rat = (df["P(0, 1)"] > df["P(0,)"]*df["P(1,)"]) & (df["P(1, 2)"] >= df["P(1,)"]*df["P(2,)"])
antecedent_rat_mean = antecedent_rat.mean()

# Defining consequent for conditional transitivity
a_fails_to_confirm_c_conditional_on_b = (df["P(0, 1, 2)"] <= ((df["P(1, 2)"]/df["P(1,)"])*df["P(0, 1)"]))

# Defining consequent for corroboration
a_fails_to_confirm_b_conditional_on_c = (df["P(0, 1, 2)"] <= ((df["P(1, 2)"]/df["P(2,)"])*df["P(0, 2)"]))
c_fails_to_confirm_b_conditional_on_a = (df["P(0, 1, 2)"] <= ((df["P(0, 1)"]/df["P(0,)"])*df["P(0, 2)"]))

# Defining OR-confirmation
probability_of_a_or_c = (df["P(0,)"] + df["P(2,)"] - df["P(0, 2)"])
probability_of_a_or_c_given_b = (df["P(0, 1)"]/df["P(1,)"] + df["P(1, 2)"]/df["P(1,)"] - df["P(0, 1, 2)"]/df["P(1,)"])

### Results for different inference patterns

# Transitivity
conj_trans_failure = (antecedent & (df["P(0, 2)"] <= df["P(0,)"]*df["P(2,)"])).mean()
cond_trans_failure = conj_trans_failure / ante_mean

# Conjunctive Transitivity
conj_conj_trans_failure = (antecedent & (df["P(0, 1, 2)"] <= df["P(0, 1)"]*df["P(2,)"])).mean()
cond_conj_trans_failure = conj_conj_trans_failure / ante_mean

# Conditional Transitivity
conj_cond_trans_failure = (antecedent & a_fails_to_confirm_c_conditional_on_b).mean()
cond_cond_trans_failure = conj_cond_trans_failure / ante_mean

# Cumulative Transitivity
conj_cum_trans_failure = (antecedent_cum & (df["P(0, 2)"] <= df["P(0,)"]*df["P(2,)"])).mean()
cond_cum_trans_failure = conj_cum_trans_failure / antecedent_cum_mean

# Agglomeration
conj_aggl_failure = (antecedent & (df["P(0, 1, 2)"] <= df["P(0, 2)"]*df["P(1,)"])).mean()
cond_aggl_failure = conj_aggl_failure / ante_mean

# Cautious Monotonicity
conj_cautm_failure = (antecedent & (df["P(0, 1, 2)"] <= df["P(0,)"]*df["P(1, 2)"])).mean()
cond_cautm_failure = conj_cautm_failure / ante_mean

# Rational Monotonicity
conj_ratm_failure = (antecedent_rat & (df["P(0, 1, 2)"] <= df["P(0,)"]*df["P(1, 2)"])).mean()
cond_ratm_failure = conj_ratm_failure / antecedent_rat_mean

# Corroboration
conj_corr_failure = (antecedent & (a_fails_to_confirm_b_conditional_on_c | c_fails_to_confirm_b_conditional_on_a)).mean()
cond_corr_failure = conj_corr_failure / ante_mean

# Amalgamation
conj_amal_failure = (antecedent & (probability_of_a_or_c_given_b  <= probability_of_a_or_c )).mean()
cond_amal_failure = conj_amal_failure / ante_mean

# Summarizing
results = pd.DataFrame([
    [conj_trans_failure, cond_trans_failure],
    [conj_conj_trans_failure, cond_conj_trans_failure],
    [conj_cond_trans_failure, cond_cond_trans_failure],
    [conj_cum_trans_failure, cond_cum_trans_failure],
    [conj_aggl_failure, cond_aggl_failure],
    [conj_cautm_failure, cond_cautm_failure],
    [conj_ratm_failure, cond_ratm_failure],
    [conj_corr_failure, cond_corr_failure],
    [conj_amal_failure, cond_amal_failure]
]).rename(columns={0: "Conjunctive Prevalence", 1: "Conditional Prevalence"})

patterns = pd.DataFrame([
['Transitivity',"If A > B and B > C, then A > C."],
['Conjunctive Transitivity', "If A > B and B > C, then A-and-B > C."],
['Conditional Transitivity', "If A > B and B > C, then A > C conditional on B."],
['Cumulative Transitivity', "If A > B and A-and-B > C, then A > C."],
['Agglomeration',"If B > A and B > C, then B > A-and-C."],
['Cautious Monotonicity',"If B > A and B > C, then B-and-C > A."],
['Rational Monotonicity',"If B > A and B does not confirm non-C, then B-and-C > A."],
['Corroboration',"If A > B and C > B, then A > B conditional on C and C > B conditional on A"],
['Amalgamation',"If A > B and C > B, then A-or-C > B."]
]).rename(columns={0: "Label", 1: "Pattern"})

pd.concat([patterns,results],axis=1).style.hide()

Label,Pattern,Conjunctive Prevalence,Conditional Prevalence
Transitivity,"If A > B and B > C, then A > C.",0.090141,0.359474
Conjunctive Transitivity,"If A > B and B > C, then A-and-B > C.",0.056355,0.224739
Conditional Transitivity,"If A > B and B > C, then A > C conditional on B.",0.110982,0.442586
Cumulative Transitivity,"If A > B and A-and-B > C, then A > C.",0.056849,0.226562
Agglomeration,"If B > A and B > C, then B > A-and-C.",0.024842,0.099068
Cautious Monotonicity,"If B > A and B > C, then B-and-C > A.",0.056435,0.225058
Rational Monotonicity,"If B > A and B does not confirm non-C, then B-and-C > A.",0.056435,0.225058
Corroboration,"If A > B and C > B, then A > B conditional on C and C > B conditional on A",0.091642,0.365460
Amalgamation,"If A > B and C > B, then A-or-C > B.",0.025064,0.099953


## Results for Absolute Confirmation

In [113]:
# Define thresholds
thresholds = [.5,.7,.9,.95,.96,.97,.98]

# Defining antecedent, conjunctive and conditional prevalence
absolute_antecedent = [((df["P(0, 1)"]/df["P(0,)"] > t) & (df["P(1, 2)"]/df["P(1,)"] > t)).mean() for t in thresholds]
conj_absolute = [((df["P(0, 1)"]/df["P(0,)"] > t) & (df["P(1, 2)"]/df["P(1,)"] > t) & (df["P(0, 2)"]/df["P(0,)"] <= t)).mean() for t in thresholds]
cond_absolute = [ num/deno for num, deno in zip(conj_absolute, absolute_antecedent)]

# Summarizing
pd.DataFrame([thresholds,conj_absolute,cond_absolute]).T.rename(columns={0: "Threshold", 1: "Conjunctive Prevalence", 2: "Conditional Prevalence"}).style.hide()

Threshold,Conjunctive Prevalence,Conditional Prevalence
0.500000,0.059070,0.236131
0.700000,0.013427,0.288176
0.900000,0.000208,0.274045
0.950000,0.000011,0.239130
0.960000,0.000007,0.350000
0.970000,0.000001,0.125000
0.980000,0.000002,0.500000
